<a href="https://colab.research.google.com/github/pozilei6/cluster-1/blob/main/Cluster_TSP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # Clustering and TSP on a set of binary arrays

Packages

In [4]:
#all packages used
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import pairwise_distances
from matplotlib import pyplot as plt
import networkx as nx
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.cm as cm

sns.set()
np.random.seed(0)

Two Packages had to be installed: mlrose and seaborn

In [ ]:
pip install mlrose

In [ ]:
pip install seaborn

# We create our binary "vectors"

In [5]:
#Y is the input Matrix for Spectral Clustering and TSP
size_y=33  #vertikal
size_x=15  #horizontal
Y=np.random.randint(2, size=(size_y, size_x))

# Now we do clustering on the rows of Y

In [9]:
sc = SpectralClustering(n_init=100)
sc.fit(Y)
labels=sc.labels_ 

# From resulting labels, we print each cluster

In [ ]:
#Group arrays into their clusters 
lbls_max = labels.max()

for k in range(lbls_max + 1):
    ind_whe_lbls_equal_k=np.where(labels == k)
    sh=(len(ind_whe_lbls_equal_k),size_x)
    SS = np.zeros(shape = sh, dtype="int")
    SS=np.array(Y[ind_whe_lbls_equal_k,:])
    print("label: ", k)
    print(SS)

# We visualize Y and Y_Cl, which is its clustered version  by labels 0, 1, ....

In [ ]:
# Sort Arrays by resulting labels 
Y_Cl=Y[labels.argsort()]

hight=max(0.4*size_y,11)
width=max(0.4*size_x,7)

# Visualize Y
fig = plt.figure(figsize=(hight,width))
plt.imshow(Y,cmap='Greys', interpolation='none')
plt.title("Plot 2D Y array")
plt.show()

# Visualize S
fig = plt.figure(figsize=(hight,width)) 
plt.imshow(Y_Cl,cmap='Greys', interpolation='none')
plt.title("Plot 2D S array")
plt.show()

# Now we do TSP on Y by using a genetic algorithm.

In [ ]:
#TSP
#Y has to be converted to a list of tuples as the argument of mlrose.TSPOpt
X=Y
list_of_tuples_X=[tuple(row) for row in X]

problem_no_fit = mlrose.TSPOpt(length = size_y, coords = list_of_tuples_X,maximize=False)
# Solve problem using the genetic algorithm
best_state, best_fitness = mlrose.genetic_alg(problem_no_fit, random_state = 2)

print('The best state found is: ', best_state)
print('The fitness at the best state is: ', best_fitness)

print("type(best_state)")
print(type(best_state))

# Finally we rearrange Y by the resulting TSP-Tour into Y_TSP and visualize both. 

In [ ]:
path=best_state
X=Y
Y_TSP=X[path,:]
# Visualize Y again
fig = plt.figure(figsize=(hight,width))
plt.imshow(Y,cmap='Greys', interpolation='none')
plt.title("Plot 2D Y array")
plt.show()
# Visualize Y_TSP
fig = plt.figure(figsize=(hight,width)) 
plt.imshow(S,cmap='Greys', interpolation='none')
plt.title("Plot 2D S array")
plt.show()

# Core issues in this code

a) Those heat maps need to be remove from those vertical and horizontal thin lines.

b) Looks like those white and black "squares" are not exactly squares. Perhaps other design    adjustements visualize the rearangement from TSP better.   

c) Same heatmaps have to be done for every cluster. At my attempt some unresolvable issues with imshow occured (something to do with dimension of input).

d) Here I used TSP. More accurately is to use a shortest-hamiltonian-path finder. By attaching one or two nodes in the TSP, this can be achieved. Perhaps there is a method in sklearn or a parameter to be adjusted for that purpose. 

e) The spectral clustering implemented through mlrose.TSPOpt() and mlrose.genetic_alg() seems to always make seven clusters. This needs to be changed as spectral clustering should always output an optimal number of clusters as it depends on the number of "small" eigenvalues. 